### 목표
- selenium 라이브러리를 통해서 네이버 지도 맛집 데이터를 수집해보기

In [352]:
# 라이브러리 불러오기
  
try :
    import os  # 운영체제 관련 작업 (폴더 생성, 삭제 등)
except ModuleNotFoundError:
    !pip install os
    
try :
    # 페이지 로딩 시간을 부여하는 라이브러리
    import time  # 시간 관련 기능 (대기, 지연)
except ModuleNotFoundError:
    !pip install time 
    
try:
    # 컴퓨터용 브라우저
    # 컴퓨터용 키보드
    # 선택지 지정방식
    from selenium import webdriver as wb
    from selenium.webdriver.common.keys import Keys
    from selenium.webdriver.common.by import By
except ModuleNotFoundError:
    !pip install selenium
    
try : 
    # 반복문 진행 상황을 확인할 수 있는 도구
    from tqdm import tqdm
except ModuleNotFoundError :
    !pip install tqdm

try :
    # 랭과 열로 이루어진 표와 같은 형태를 다룰 수 있는 라이브러리
    import pandas as pd 
except ModuleNotFoundError :
    !pip install pandas

print(f'필요한 라이브러리 import 완료')

필요한 라이브러리 import 완료


In [353]:
# 정보

URL = 'https://map.naver.com/p?c=15.00,0,0,0,dh'

# 메인폴더 위치
ORIGIN_URL = "C:\\Users\\smhrd1\\Documents\\GitHub\\Crawling\\Study"

# 지도의 검색창 위치
MAP_IMPUT = 'input.input_search'

# 지도에서 식당의 이름
MAP_NAME = 'TYaxT'

# 지도에서 end를 위한 위치 선택
MAP_SCROLL = 'Ryr1F'



In [354]:
driver = wb.Chrome()
driver.get(URL) 
time.sleep(3)

In [355]:
map_input = driver.find_element(By.CSS_SELECTOR, MAP_IMPUT)  
for i in range(10) :
    map_input.send_keys(Keys.BACK_SPACE)
map_input.send_keys('순천 맛집')
map_input.send_keys(Keys.ENTER)

time.sleep(2)   

In [356]:
origin_get_All_List = driver.find_elements(By.CSS_SELECTOR, MAP_NAME)

In [357]:
# iframe 창 전환하는 함수

driver.switch_to.frame('searchIframe') # id 값만 

In [358]:
while(True) :
    map_scr = driver.find_element(By.CLASS_NAME, MAP_SCROLL) 
    origin_len = len(driver.find_elements(By.CSS_SELECTOR, 'li'))
    map_scr.send_keys(Keys.END) 
    time.sleep(0.5)
    map_scr.send_keys(Keys.END) 
    time.sleep(0.5)
    later_len = len(driver.find_elements(By.CSS_SELECTOR, 'li'))
    if(origin_len == later_len) :
        print("페이지의 끝은지 검사 중") 
        map_scr.send_keys(Keys.END)
        print(len(driver.find_elements(By.CSS_SELECTOR, 'li')))
        time.sleep(1) 
        check_len = len(driver.find_elements(By.CSS_SELECTOR, 'li'))
        if(later_len == check_len) :
            print("페이지가 끝났습니다. ")
            break
        else :
            print("페이지가 끝나지 않아서 다시 스크롤을 시작합니다. ")

페이지의 끝은지 검사 중
66
페이지가 끝났습니다. 


In [359]:
get_All_List = driver.find_elements(By.CLASS_NAME, MAP_NAME)

In [360]:
name_list = [i.text for i in get_All_List]
print(len(name_list))
name_list

66


['유유샤브 순천점',
 '잉글우드',
 '밥꽃이야기 들마루',
 '대원식당',
 '거목순대국밥',
 '덕수파스타 순천 직영점',
 '참조은시골집 한정식',
 '도원경',
 '순천만전라도밥상',
 '순천 한정식 명궁관',
 '별미정 순천조례점',
 '순천 건봉국밥',
 '하남돼지집 순천신대점',
 '최군네 숯불구이',
 '조훈모과자점 죽도봉점',
 '희야 소머리국밥 설렁탕 도가니탕',
 '힐링포차',
 '아삐에노',
 '꽃잎 한정식',
 '모토이시 순천점',
 '에스가든청담 브런치가든 순천점',
 '꾸석지돌판한우 순천조례점',
 '벽오동',
 '어나더키친 순천점',
 '풍미통닭',
 '신화정',
 '최대감벌교꼬막',
 '서식',
 '밀림슈퍼',
 '카페집',
 '육지향 정육식당 신대점',
 '오리명가',
 '화월당',
 '구이대장쪽갈비 신대점',
 '순천갈대장어 연향점',
 '남흥회관 순천시청본점',
 '금빈회관',
 '봉화식당',
 '향토정',
 '한우식껍',
 '주복',
 '하고스',
 '힉',
 '히바치텐',
 '육안 순천고기집',
 '고데레',
 '순천 산들담은 순천본점',
 '수련산방',
 '여수갓육고 순천본점',
 '태경원조동경낙지',
 '나루',
 '홍성준양푼갈비',
 '옛날할머니순대국밥',
 '양지쌈밥',
 '괴목국밥',
 '향촌식당',
 '지리산순한한우명품관 본점',
 '순천만정문식당',
 '제일식당',
 '광춘원',
 '참이맛감자탕&순대국 순천법원점',
 '슬로우프로젝트 즐거운식탁',
 '순천만 일번가',
 '육사 순희집밥',
 '코우텐동',
 '바바베이커스']

In [362]:
store_info = []

# 첫 번째 가게를 클릭  
for i in  tqdm( range(len(name_list))) :
    get_All_List[i].click()
    
    time.sleep(1)
    
    driver.switch_to.default_content()
    driver.switch_to.frame('entryIframe')
    check = driver.find_element(By.CLASS_NAME, 'LDgIH') 
    store_info.append(check.text)
    
    driver.switch_to.default_content()
    driver.switch_to.frame('searchIframe')
    
    time.sleep(0.2)

  0%|                                                                                           | 0/66 [00:00<?, ?it/s]

전남 순천시 왕지4길 10-16 1층


  2%|█▎                                                                                 | 1/66 [00:01<01:56,  1.80s/it]

전남 순천시 해룡면 남가1길 2 잉글우드


  3%|██▌                                                                                | 2/66 [00:03<01:55,  1.80s/it]

...

전남 순천시 팔마4길 37 1층


 21%|█████████████████▍                                                                | 14/66 [00:24<01:30,  1.75s/it]

전남 순천시 봉화로 46 조훈모과자점


 21%|█████████████████▍                                                                | 14/66 [00:25<01:36,  1.85s/it]


KeyboardInterrupt: 

In [ ]:
address_list = [i.text for i in store_info] 
address_list

In [ ]:
driver.close()